## FineTuning LLM with Model-As-A-Service - TextGeneration

This sample shows how to create a standalone FineTuning job to fine tune a model to summarize the input text provided in the training data.
Model-As-A-Service means user simply provides input data to finetune and all the resources are provided by system.

#### Training data
We use the [samsum]((https://huggingface.co/datasets/samsum)) dataset. For simplicity we preprocessed the data and have only fraction of the data put here along with the notebook. The dataset, suitable for:
* Supervised fine-tuning (sft).
* Generation ranking (gen).

Note that sample data is only useful for demo purposes. Also terms 'text-generation/text-completion' are used interchangeably and mean same task of finetuning.

#### Model
We will use the Meta-Llama-3-8B model to show how user can finetune a model for Text-Generation task. If you opened this notebook from a specific model card, remember to replace the specific model name. 

#### Outline
1. Setup pre-requisites
2. Pick a model to fine-tune.
3. Create training and validation datasets.
4. Configure the fine tuning job.
5. Submit the fine tuning job.
6. Deploy the fine tuned model for real time inference.

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name

**Install dependencies by running below cell. This is not an optional step if running in a new environment.**

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity

%pip install mlflow
%pip install azureml-mlflow

### Create AzureML Workspace connections

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential=credential)
except:
    workspace_ml_client = MLClient(
        credential,
        subscription_id="<SUBSCRIPTION_ID>",
        resource_group_name="<RESOURCE_GROUP>",
        workspace_name="<WORKSPACE_NAME>",
    )

# the models, fine tuning pipelines and environments are available in various AzureML system registries,
# Example: Phi family of models are in "azureml", Llama family of models are in "azureml-meta" registry.
registry_ml_client = MLClient(credential, registry_name="azureml-meta")

# Get AzureML workspace object.
workspace = workspace_ml_client._workspaces.get(workspace_ml_client.workspace_name)
workspace.id

### 2. Pick a model to fine tune

`Meta-Llama-3-8B` is a 3.8B parameters, lightweight, state-of-the-art open model built by `Meta`. The model belongs to the Meta model family. You can browse these models in the Model Catalog in the Azure AI Studio, filtering by the `text-completion` task.

Note the model id property of the model. This will be passed as input to the fine tuning job. This is also available as the `Asset ID` field in model details page in Azure AI Studio Model Catalog.

In [ ]:
model_name = "Meta-Llama-3-8B"
model_to_finetune = registry_ml_client.models.get(model_name, label="latest")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
        model_to_finetune.name, model_to_finetune.version, model_to_finetune.id
    )
)

#### Use either compute cluster or provide instance type which is compatible with below list

In [ ]:
model_to_finetune.properties["finetune-recommended-sku"]

### 3. Sample data

The text-generation dataset is stored in this format

    {
        "text": "Summarize the dialog.\n<dialog>: Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)\n<summary>: ",
        "ground_truth":"Amanda baked cookies and will bring Jerry some tomorrow.",
    }

#### Create data inputs

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data

dataset_version = "1"
train_dataset_name = "text_generation_train_small"
try:
    train_data_asset = workspace_ml_client.data.get(
        train_dataset_name, version=dataset_version
    )
    print(f"Dataset {train_dataset_name} already exists")
except:
    print("creating dataset")
    train_data = Data(
        path=f"./train.jsonl",
        type=AssetTypes.URI_FILE,
        description="Training dataset",
        name=train_dataset_name,
        version="1",
    )
    train_data_asset = workspace_ml_client.data.create_or_update(train_data)

In [ ]:
train_data_asset.id

In [ ]:
from azure.ai.ml.entities import Data

dataset_version = "1"
validation_dataset_name = f"text_generation_validation_small"
try:
    validation_data_asset = workspace_ml_client.data.get(
        validation_dataset_name, version=dataset_version
    )
    print(f"Dataset {validation_dataset_name} already exists")
except:
    print("creating dataset")
    validation_data = Data(
        path=f"./validation.jsonl",
        type=AssetTypes.URI_FILE,
        description="Validation dataset",
        name=validation_dataset_name,
        version="1",
    )
    validation_data_asset = workspace_ml_client.data.create_or_update(validation_data)

In [ ]:
validation_data_asset.id

#### Create marketplace subscription for 3P models
**Note:** Skip this step for 1P(Microsoft) models that are offered on Azure. Example: Phi family of models

In [ ]:
model_id_to_subscribe = "/".join(model_to_finetune.id.split("/")[:-2])
print(model_id_to_subscribe)

normalized_model_name = model_name.replace(".", "-")

In [ ]:
from azure.ai.ml.entities import MarketplaceSubscription


subscription_name = f"{normalized_model_name}-sub"

marketplace_subscription = MarketplaceSubscription(
    model_id=model_id_to_subscribe,
    name=subscription_name,
)

# note: this will throw exception if the subscription already exists or subscription is not required (for example, if the model is not in the marketplace like Phi family)
try:
    marketplace_subscription = (
        workspace_ml_client.marketplace_subscriptions.begin_create_or_update(
            marketplace_subscription
        ).result()
    )
except Exception as ex:
    print(ex)

### 3. Submit the fine tuning job using the the model and data as inputs
 
Create FineTuning job using all the data that we have so far.

#### Define finetune parameters

##### There are following set of parameters that are required.

1. `model` - Base model to finetune.
2. `training_data` - Training data for finetuning the base model.
3. `validation_data` - Validation data for finetuning the base model.
4. `task` - FineTuning task to perform. eg. TEXT_COMPLETION for text-generation/text-generation finetuning jobs.
5. `outputs`- Output registered model name.

##### Following parameters are optional:

1. `hyperparameters` - Parameters that control the FineTuning behavior at runtime.
2. `name`- FineTuning job name
3. `experiment_name` - Experiment name for FineTuning job.
4. `display_name` - FineTuning job display name.

In [ ]:
from azure.ai.ml.finetuning import FineTuningTaskType, create_finetuning_job
import uuid

guid = uuid.uuid4()
short_guid = str(guid)[:8]
display_name = f"{model_name}-display-name-{short_guid}"
name = f"finetuned-model-{short_guid}"
output_model_name_prefix = f"finetuned-model-{short_guid}"
experiment_name = f"finetuning-llm"
compute = "nc96-lowpri-eastus"

finetuning_job = create_finetuning_job(
    task=FineTuningTaskType.TEXT_COMPLETION,
    training_data=train_data_asset.id,
    validation_data=validation_data_asset.id,
    hyperparameters={
        "per_device_train_batch_size": "1",
        "learning_rate": "0.00002",
        "num_train_epochs": "1",
    },
    model=model_to_finetune.id,
    display_name=display_name,
    name=name,
    experiment_name=experiment_name,
    # compute=compute,
    instance_types=["Standard_ND96amsr_A100_v4", "Standard_E4s_v3"],
    tags={"foo_tag": "bar"},
    properties={"my_property": "my_value"},
    output_model_name_prefix=output_model_name_prefix,
)

In [ ]:
created_job = workspace_ml_client.jobs.create_or_update(finetuning_job)
workspace_ml_client.jobs.get(created_job.name)

#### Wait for the above job to complete successfully

In [ ]:
status = workspace_ml_client.jobs.get(created_job.name).status

import time

while True:
    status = workspace_ml_client.jobs.get(created_job.name).status
    print(f"Current job status: {status}")
    if status in ["Failed", "Completed", "Canceled"]:
        print("Job has finished with status: {0}".format(status))
        break
    else:
        print("Job is still running. Checking again in 30 seconds.")
        time.sleep(30)

In [ ]:
if status in ["Failed", "Canceled"]:
    print(
        "Job did not finish successfully. So no model to deploy. JobStatus: {0}".format(
            status
        )
    )

In [ ]:
finetune_model_name = created_job.outputs["registered_model"]["name"]
finetune_model_name

In [ ]:
endpoint_name = f"{normalized_model_name}-ft-{short_guid}"  # Name must be unique
model_id = f"azureml://locations/{workspace.location}/workspaces/{workspace._workspace_id}/models/{finetune_model_name}/versions/1"

#### 4. Create serverless endpoint using the finetuned model

In [ ]:
from azure.ai.ml.entities import ServerlessEndpoint

serverless_endpoint = ServerlessEndpoint(name=endpoint_name, model_id=model_id)

created_endpoint = workspace_ml_client.serverless_endpoints.begin_create_or_update(
    serverless_endpoint
).result()

In [ ]:
endpoint = workspace_ml_client.serverless_endpoints.get(endpoint_name)
endpoint_keys = workspace_ml_client.serverless_endpoints.get_keys(endpoint_name)
auth_key = endpoint_keys.primary_key

In [ ]:
import requests

url = f"{endpoint.scoring_uri}/v1/completions"

payload = {
    "max_tokens": 1024,
    "text""Summarize the dialog.\n<dialog>: David: Hi, do you have a minute?\nAngela: Hi, yes, tell me\nDavid: Could you tell me what happened between Pamela and Maggie? They're not talking to each other\nAngela: I know, the reason is pretty weird\nDavid: I guessed so\nAngela: They argued because of their university courses\nDavid: Why?\nAngela: Some of the students were not satisfied with one of the language courses and decided to tell the teacher\nDavid: Oh\nAngela: Pamela and another student talked to the teacher on behalf of the group\nDavid: Ok, and...?\nAngela: And Maggie and a number of others were pissed off because they said the teacher would be angry at all of them and it would mean trouble for the whole group\nDavid: Ouch\nAngela: Yes, so Pam and Maggie argued and now they're not friends anymore\nDavid: I hope they will reconcile soon\nAngela: Yeah\n<summary>: ",
}
headers = {"Content-Type": "application/json", "Authorization": f"{auth_key}"}

response = requests.post(url, json=payload, headers=headers)

response.json()